
# EEG Artifact removal using ATAR

ATAR: Automatic and Tunable Artifact Removal Algorithm

**ATAR Algorithm -  Automatic and Tunable Artifact Removal Algorithm for EEG Signal.** 

The algorithm is based on wavelet packet decomposion (WPD), the full description of algorithm can be 
found here [Automatic and Tunable Artifact Removal Algorithm for EEG](https://doi.org/10.1016/j.bspc.2019.101624) 
from the article [1]. Figure 1 shows the the block diagram and operating mode of filtering.


.. figure:: https://raw.githubusercontent.com/spkit/spkit.github.io/master/assets/images/ATAR_BD_FilteringMode_1.png
  :align: center

The algorithm is applied on the given multichannel signal X (n,nch), window wise and reconstructed 
with overall add method. The defualt window size is set to 1 sec (128 samples). For each window, 
the threshold $   heta_lpha$ is computed and applied to filter the wavelet coefficients.

There is manily one parameter that can be tuned $eta$ with different operating modes and other settings.
Here is the list of parameters and there simplified meaning given:

Parameters:
* $eta:$ This is a main parameter to tune, highher the value, more aggressive the algorithm to 
remove the artifacts. By default it is set to 0.1. $eta$ is postive float value.

* ***OptMode***: This sets the mode of operation, which decides hoe to remove the artifact. By default 
it is set to 'soft', which means Soft Thresholding, in this mode, rather than removing 
the pressumed artifact, it is suppressed to the threshold, softly. `OptMode='linAtten'`, 
suppresses the pressumed artifact depending on how far it is from threshold. Finally, 
the most common mode - Elimination (OptMode='elim'), which remove the pressumed artifact.

    * Soft Thresholding and Linear Attenuation require addition parameters to set 
    the associated thresholds which are by default set to bf=2, gf=0.8. 

* ***wv=db3***: Wavelet funtion, by default set to db3, could be any of 
  ['db3'.....'db38', 'sym2.....sym20', 'coif1.....coif17', 'bior1.1....bior6.8', 'rbio1.1...rbio6.8', 'dmey']

* *$k_1$*, *$k_2$*: Lower and upper bounds on threshold :math:` heta_lpha.
* ***IPR=[25,75]***: interpercentile range, range used to compute threshold


In [ ]:
# Importing libraries/spkit
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import spkit as sp
from spkit.data import load_data
print('spkit version :', sp.__version__)



# 14 channel EEG sample signas

X,ch_names = load_data.eegSample()
fs = 128

Xf = sp.filter_X(X,band=[0.5], btype='highpass',fs=fs,verbose=0)
print(Xf.shape)

Artifact removal using ATAR



In [ ]:
XR = sp.eeg.ATAR(Xf.copy(),verbose=0)
print(XR.shape)

t = np.arange(Xf.shape[0])/fs
plt.figure(figsize=(12,8))
plt.subplot(211)
plt.plot(t,XR+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('XR: Corrected Signal')
plt.subplot(212)
plt.plot(t,(Xf-XR)+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf - XR: Difference (removed signal)')
plt.suptitle('ATAR: Soft Thresholding (default mode)')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12,5))
plt.plot(t,Xf+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf: 14 channel - EEG Signal (filtered)')
plt.tight_layout()
plt.show()

Artifact removal using ATAR -  Linear Attenuation Mode



In [ ]:
XR = sp.eeg.ATAR(Xf.copy(),verbose=0,OptMode='linAtten')
print(XR.shape)

t = np.arange(Xf.shape[0])/fs
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(t,XR+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('XR: Corrected Signal')
plt.subplot(122)
plt.plot(t,(Xf-XR)+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf - XR: Difference (removed signal)')
plt.suptitle('ATAR: Linear Attenuation Mode')
plt.tight_layout()
plt.show()

Artifact removal using ATAR - Elimination Mode



In [ ]:
XR = sp.eeg.ATAR(Xf.copy(),verbose=0,OptMode='elim')
print(XR.shape)

t = np.arange(Xf.shape[0])/fs
plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(t,XR+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('XR: Corrected Signal')
plt.subplot(122)
plt.plot(t,(Xf-XR)+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf - XR: Difference (removed signal)')
plt.suptitle('ATAR: Elimination Mode')
plt.tight_layout()
plt.show()

Tuning $\beta$ with 'soft' : Controlling the aggressiveness



In [ ]:
betas = np.r_[np.arange(0.01,0.1,0.02), np.arange(0.1,1, 0.1)].round(2)

t = np.arange(Xf.shape[0])/fs

for b in betas:
    XR = sp.eeg.ATAR(Xf.copy(),verbose=0,beta=b,OptMode='soft')
    XR.shape

    plt.figure(figsize=(15,5))
    plt.subplot(121)
    plt.plot(t,XR+np.arange(-7,7)*200)
    plt.xlim([t[0],t[-1]])
    plt.xlabel('time (sec)')
    plt.yticks(np.arange(-7,7)*200,ch_names)
    plt.grid()
    plt.title('XR: Corrected Signal: '+r'$\beta=$' + f'{b}')
    plt.subplot(122)
    plt.plot(t,(Xf-XR)+np.arange(-7,7)*200)
    plt.xlim([t[0],t[-1]])
    plt.xlabel('time (sec)')
    plt.yticks(np.arange(-7,7)*200,ch_names)
    plt.grid()
    plt.title('Xf - XR: Difference (removed signal)')
    plt.suptitle('ATAR: ' +r'$\beta=$' + f'{b}')
    plt.tight_layout()
    plt.show()

On smaller segment



In [ ]:
Xf1 = Xf[128*10:128*14].copy()

t = np.arange(Xf1.shape[0])/fs
plt.figure(figsize=(12,5))
plt.plot(t,Xf1+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf: 14 channel - EEG Signal (filtered)')
plt.tight_layout()
plt.show()


XR1 = sp.eeg.ATAR(Xf1.copy(),verbose=0)
print(XR1.shape)


plt.figure(figsize=(15,5))
plt.subplot(121)
plt.plot(t,XR1+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('XR: Corrected Signal')

plt.subplot(122)
plt.plot(t,(Xf1-XR1)+np.arange(-7,7)*200)
plt.xlim([t[0],t[-1]])
plt.xlabel('time (sec)')
plt.yticks(np.arange(-7,7)*200,ch_names)
plt.grid()
plt.title('Xf - XR: Difference (removed signal)')
plt.tight_layout()
plt.show()